# Spaceshot Loop Investigation

This document investigates the Spaceshot Loop using Lagrangian dynamics. We hope to recreate the Spaceshot Loop without accounting for aerodynamic terms to avoid the headaches that would follow from that. Simply put, this is our lazy first shot because air is hard. For a no-prior-experience introduction to the Spaceshot Loop and how we model it here, please see Chandler's [_Spaceshot Loop Dynamics_ guide](https://drive.google.com/file/d/1J3nmpmz2pRV-1YVPBqL_L5fdTEglLyGC/view?usp=sharing). First, let's import everything we need:

In [3]:
from sympy import *
from sympy.physics.mechanics import *

Now, recall the Spaceshot Lagrangian (that has a nice ring to it!):

$$L = \frac{1}{2}m(\dot{x}^2 + \dot{y}^2 + \dot{z}^2) + \frac{1}{2} I_1 (\dot{\phi} \sin \theta + \dot{\theta}) + \frac{1}{2} I_3 (\dot{\phi} \cos \theta + \dot{\psi})^2$$

We want to represent this in code.

For information on Sympy, see the Introduction section at [this link](https://docs.sympy.org/latest/tutorial/index.html). For using SymPy for Lagrangian mechanics, see [here](https://docs.sympy.org/latest/modules/physics/mechanics/lagrange.html).

Here we go:

In [37]:
m, g, t = symbols('m, g, t')
I1, I3, T = symbols('I1, I3, T')
x, y, z = dynamicsymbols('x, y, z')
phi, theta, psi = dynamicsymbols('phi, theta, psi')
xdot, ydot, zdot = dynamicsymbols('x, y, z', level=1)
phidot, thetadot, psidot = dynamicsymbols('phi, theta, psi', level=1)

L = (m/2 * (xdot ** 2 + ydot ** 2 + zdot ** 2)
     + I1/2 * (phidot * sin(theta) + thetadot)
     + I3/2 * (phidot * cos(theta) + psidot)**2)

That wasn't bad. Now let's get our Euler-Lagrange equations. Note that the expression below only gives the left hand sides of our Euler-Lagrange equations. Usually the right hand side is all zeros, unless we have external forces acting on our rocket--and we do, so we'll add those forces in in a minute.

Also, rather than using the builtin `LagrangesMethod` function, we just take the derivatives by hand. This way, you can see what's going on under the hood (this is just Lagrange's equation from the notes) and I get to avoid all the hairy stuff involved in that SymPy function. We're going to pretend it's a win-win.

The list below, called `equations`, has the left-hand sides of all the equations we want to solve. Rather than having the right hand side be the force our rocket experiences (like we claimed it would be above), we subtract over those forces we were going to put on the right-hand side, and make all the right-hand sides zero.

This is just going from
$$\text{some garbage} = \text{force}$$
to
$$\text{some garbage} - \text{force} = 0.$$
Simple enough!

Also, our first of two forces are gravity (recall $\hat{z}$ is just the unit vector in the $z$ direction):
$${F} = -mg \hat{z}.$$
The other is thrust:
$$\vec{T} = T \sin \phi \sin \theta \hat{x} + T \cos \phi \sin \theta \hat{y} + T \cos \theta \hat{z}$$
where $T$ is the magnitude of thrust.

Don't worry about the sines and cosines here: they're just here to rotate the thrust from straight up and down to lining up with the rocket (I probably couldn't rederive them if you gave me a piece of paper and a week, but to be fair I'm not great with geometry in the first place and generally somewhat lazy).



In [82]:
thrust_force_x = T * sin(phi) * sin(theta)
thrust_force_y = T * cos(phi) * sin(theta)
thrust_force_z = T * cos(theta)
gravity_force_z = -g * z

equations = [L.diff(xdot).diff(t) - L.diff(x) - thrust_force_x,
             L.diff(ydot).diff(t) - L.diff(y) - thrust_force_y,
             L.diff(zdot).diff(t) - L.diff(z) - thrust_force_z - gravity_force_z,
             L.diff(phidot).diff(t) - L.diff(phi),
             L.diff(thetadot).diff(t) - L.diff(theta),
             L.diff(psidot).diff(t) - L.diff(psi)]

Ah, I forgot to mention: most numerical differential equation solvers in Python like first-order equations (i.e. highest derivative is first order. But all of our equations are second order! For instance, our equation for $\phi$ gives

In [76]:
equations[3]

I1*cos(theta(t))*Derivative(theta(t), t)/2 - I3*(cos(theta(t))*Derivative(phi(t), t) + Derivative(psi(t), t))*sin(theta(t))*Derivative(theta(t), t) + I3*(-sin(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) + cos(theta(t))*Derivative(phi(t), (t, 2)) + Derivative(psi(t), (t, 2)))*cos(theta(t))

Those 2s in those second derivatives have got to go! So, we perform the usual dumb trick: introduce a new variable for each existing one that we set equal to the existing one's derivative. For shorthand, we use the notation `a'` to represent the derivative of a variable `a'`. Cool. Cool cool cool.

So for `x`, we introduce `a` (a completely new variable, unassociated with `x`!) and replace the derivatives of `x` with `a`. Then, so it actually becomes that derivative, we set `a` equal to `x'` in a new equation. As an example, `3x'' + 5 = 0` would become `3a' + 5 = 0` and `x' = a`. One equation now becomes two smaller-order ones, together with the same solutions as the original equation. 

I would name these better (like maybe suggestively calling `a` something like `x_diff` or something) but as soon as we tried to pretty-print that all the subscripts it would make it a mess (I may or may not have tried that already).

In [84]:
a, b, c, d, e, f = dynamicsymbols('a, b, c, d, e, f')
diffs = {x: a, y: b, z: c, phi: d, theta: e, psi: f}

new_equations = [equation for equation in equations]
equation_count = len(equations)
for original, diff in diffs.items():
    for i in range(equation_count):
        new_equations[i] = new_equations[i].subs(original.diff(t), diff)
    new_equations.append(original.diff(t) - diff)

[-T*sin(phi(t))*sin(theta(t)) + m*Derivative(a(t), t),
 -T*sin(theta(t))*cos(phi(t)) + m*Derivative(b(t), t),
 -T*cos(theta(t)) + g*z(t) + m*Derivative(c(t), t),
 I1*e(t)*cos(theta(t))/2 - I3*(d(t)*cos(theta(t)) + f(t))*e(t)*sin(theta(t)) + I3*(-d(t)*e(t)*sin(theta(t)) + cos(theta(t))*Derivative(d(t), t) + Derivative(f(t), t))*cos(theta(t)),
 -I1*d(t)*cos(theta(t))/2 + I3*(d(t)*cos(theta(t)) + f(t))*d(t)*sin(theta(t)),
 I3*(-2*d(t)*e(t)*sin(theta(t)) + 2*cos(theta(t))*Derivative(d(t), t) + 2*Derivative(f(t), t))/2,
 -a(t) + Derivative(x(t), t),
 -b(t) + Derivative(y(t), t),
 -c(t) + Derivative(z(t), t),
 -d(t) + Derivative(phi(t), t),
 -e(t) + Derivative(theta(t), t),
 -f(t) + Derivative(psi(t), t)]